In [1]:
import pandas as pd
from snappy import *
from cytoolz import *
from braidgenerator import *
import spherogram as s
import multiprocessing as m

In [2]:
numBraids = 10000; #generate 10k braids of each length
maxLen    = 15;
maxGen    = 15;
numSteps  = 10;
data = dict()
for i in range(3,maxLen+1):
    x = MarkovChain(braidword=[1,2,-2-1], maxgen=maxGen, maxlen=i)
    res = None #Loop until it works. Random algorithms are weird!
    while res is None:
        try:
            x.model(num_braidreps=numBraids, msteps=numSteps)
            res = True
        except:
            pass
    data[i] = pd.Series(x.braidreps()).map(lambda b: b.word)

In [3]:
bs = ( pd.DataFrame([(k, v) for k in data.keys() for v in data[k]], columns=["length", "braid"])
         .assign(braid = lambda df: df.braid.map(lambda v: tuple(filter(lambda z: z!=0, v)))) #removes trivial letters 0 = id and convers to tuples, which are hashable
         .drop_duplicates('braid')  #Drop duplicate entries.
         .assign(braid = lambda df: df.braid.map(list)) #convert back to lists (ie words)
         .assign(braid = lambda df: df.braid.replace(to_replace=[],value=pd.NA)) #replace empty braids with NA
         .dropna() # drop empty braids
         .sort_values(by="length", ascending=False) #sort by lengthof braid, descending
         .reset_index(drop=True) #reset the indexing to be nice.
     )
# print(len(bs))
# 100 - len(bs)/sum(map(len, data)) * 100

In [4]:
bs.length.value_counts(sort=False)

3        6
4       24
5      263
6     1321
7     4563
8     9410
9     8790
10    8944
11    8998
12    9067
13    9222
14    9206
15    9288
Name: length, dtype: int64

In [5]:
t = bs[bs.length<= 5].braid
links = [s.Link(braid_closure=braid) for braid in t]
print(list(take(5, [L.PD_code() for L in links])))

[[(8, 0, 9, 9), (1, 6, 2, 7), (5, 3, 6, 2), (4, 4, 5, 3), (7, 1, 8, 0)], [(8, 3, 9, 4), (5, 7, 6, 6), (2, 0, 3, 9), (4, 8, 5, 7), (1, 1, 2, 0)], [(5, 5, 6, 4), (2, 7, 3, 8), (8, 2, 9, 1), (6, 4, 7, 3), (9, 1, 0, 0)], [(2, 7, 3, 8), (8, 2, 9, 1), (6, 4, 7, 3), (5, 5, 6, 4), (9, 1, 0, 0)], [(6, 2, 7, 1), (8, 7, 9, 8), (3, 3, 4, 2), (4, 0, 5, 9), (5, 0, 6, 1)]]


In [ ]:
# links[0].view() #this stalls out jupyter.
# %gui tk

If the fundamental group of $\pi_1(S^3\setminus K) \cong \mathbb{Z}$, we can be 100% certain that K is the unknot

In [6]:
# def getFG(l: s.Link)-> FundamentalGroup:
#     return L.exterior().fundamental_group()
for L in take(5, links):
    print(L.exterior().fundamental_group())

Generators:
   a
Relators:
   
Generators:
   a
Relators:
   
Generators:
   a
Relators:
   
Generators:
   a
Relators:
   
Generators:
   a
Relators:
   


In [27]:
mylink = links[0].connected_sum(links[1])

In [28]:
mylink.view()
%gui tk

Your new Plink window needs an event loop to become visible.
Type "%gui tk" below (without the quotes) to start one.



In [30]:
mylink.braid_word()

[-1,
 2,
 1,
 -2,
 3,
 2,
 4,
 -5,
 6,
 7,
 -8,
 -7,
 9,
 -6,
 8,
 10,
 5,
 7,
 -9,
 -4,
 6,
 8,
 3,
 5,
 7,
 6,
 -7,
 -6,
 -8,
 -5,
 -7,
 9,
 6,
 -8,
 -10,
 7,
 -9,
 8]